# Resumen clase anterior


<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="35%" align="left">
<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_matriz.jpg" width="55%" align="rigth"/>

Calcular el valor de los flujos metabólicos usando numpy.

1. Asumiendo que el flujo limitante es E1=10 [mmol/gDW/h] y que el organismo maximiza la producción de CH4.
2. Igual que 1 pero maximizando la producción de F420.

In [28]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

# E1,E2,E3,E4,E5,R1,R2,R3

A = np.array([
[1,0,0,0,0,-1,0,0],
[0,1,0,0,0,-1,-1,0],
[0,0,0,0,0,1,-1,1],
[0,0,0,-2,0,-1,1,1],
[0,0,-1,0,0,0,1,-1],
[0,0,0,0,-1,0,0,1]])        

b = np.array([0,0,0,0,0,0])
c = np.array([0,0,1,0,0,0,0,0])

LB=[10,0,0,0,0,0,0,0]       # [0,0,0,0,0,0]
UB=[10,1000000,1000000,1000000,1000000,1000000,1000000,1000000]     # ya sabes la idea

res = linprog(-c, A_eq=A, b_eq=b, bounds=list(zip(LB,UB)))
print('Optimal value:', res.fun, '\nX:', res.x)


Optimal value: -10.000000001210326 
X: [1.00000000e+01 5.00013871e+05 1.00000000e+01 4.99993871e+05
 4.99993871e+05 1.00000000e+01 5.00003871e+05 4.99993871e+05]


In [33]:
list(zip(LB,UB))

[(10, 10),
 (0, 1000000),
 (0, 1000000),
 (0, 1000000),
 (0, 1000000),
 (0, 1000000),
 (0, 1000000),
 (0, 1000000)]

# Clase 6

Pasar lista

## Contenidos

* Resolver problemas de ingeniería metabólica a escala genómica
* Introducción a cobrapy



# Introducción a cobra


Para modelar el metabolismo celular a escala genómica utilizaremos la librería ```cobra```. La documentación oficial de cobra se encuentra disponible [aquí](https://cobrapy.readthedocs.io/en/latest/index.html). Hoy veremos los comandos escenciales de cobra y construiremos nuestra propia red metábolica.

## Instalación


Para instalar ```cobra``` debes abrir el terminal de anaconda (si usas Windows, busca Anaconda Prompt ) y escribir en el siguiente comando:

```python
pip install cobra --user
```

Para probar que la librería se instlado correctamente escribe el siguiente comando en la próxima celda:

```python
import cobra
```

Si no hay mensajes de error, entonces ya tienes una versión funcional de cobra.

## Comandos básicos


Comezaremos cargado ```cobra``` y determinando que version está instalada (esto es útil para consultar la documentación adecuada; diferentes versiones de cobra tienen diferentes documentaciones)

In [34]:
import cobra
cobra.__version__

'0.22.1'

## Redes metabólicas a escala genómica


En Inglés Genome-scale metabolic networks son redes metabólicas que incluyen todas las reacciones metabólicas presentes en el genoma del organismo bajo estudio.

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_eColiMetabolis.png" width="45%" align="left">

En el caso de *Escherichia coli* la red consiste en  reacciones y metabolitos.

["Fuente KEGG"](https://www.genome.jp/kegg-bin/show_pathway?eco01100+M00125)

In [35]:
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("ecoli")

Veamos cuantas reacciones y metabolitos tiene "ecoli"

In [36]:
print( "Número de reacciones: " , len(model.reactions) )
print( "Número de metabolitos: ", len(model.metabolites) )

Número de reacciones:  2583
Número de metabolitos:  1805


Podemos inspeccionar reacciones individualmente, por ejemplo la reacción 30:

In [37]:
print( model.reactions[30].reaction )

5mtr_e --> 


Ver las primeras 10 reacciones:

In [38]:
for i in range(10):
    print( model.reactions[i] )

DM_4crsol_c: 4crsol_c --> 
DM_5drib_c: 5drib_c --> 
DM_aacald_c: aacald_c --> 
DM_amob_c: amob_c --> 
DM_mththf_c: mththf_c --> 
DM_oxam_c: oxam_c --> 
BIOMASS_Ec_iJO1366_WT_53p95M: 0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 54.119975 atp_c + 0.000116 bmocogdp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 48.752916 h2o_c + 0.000223 hemeO_c + 0.092056 his__

## Nota

¿Cuál es tu interpretación de la reacción número 7?

También podemos identificar una reacción por su nombre:

In [41]:
pgi = model.reactions.get_by_id("PGI")
print( pgi )
print( pgi.reaction )
print( pgi.name )


print(pgi.lower_bound, "< pgi <", pgi.upper_bound)

udcpdp = model.metabolites.get_by_id("udcpdp_c")
print( udcpdp.reactions )
print( udcpdp.formula )

PGI: g6p_c <=> f6p_c
g6p_c <=> f6p_c
Glucose-6-phosphate isomerase
-1000.0 < pgi < 1000.0
frozenset({<Reaction UDCPDPS at 0x7f0099d20d00>, <Reaction BIOMASS_Ec_iJO1366_WT_53p95M at 0x7f009ac0ad30>, <Reaction MPTG at 0x7f009a305b80>, <Reaction MPTG2 at 0x7f009a305ca0>, <Reaction UDCPDP at 0x7f0099d20fa0>, <Reaction BIOMASS_Ec_iJO1366_core_53p95M at 0x7f009ac1ffd0>})
C55H89O7P2


Podemos verificar el balance de masa de una reacción:

In [42]:
print(  "Balance de masa original" )
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

print(  "Veamos que pasa con el balance de masa si ahora agregamos un metabolito a la reacción" )
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

Balance de masa original
g6p_c <=> f6p_c
{}
Veamos que pasa con el balance de masa si ahora agregamos un metabolito a la reacción
g6p_c + h_c <=> f6p_c
{'charge': -1.0, 'H': -1.0}


In [50]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): 1,model.metabolites.get_by_id("atp_c"): 1})
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

g6p_c <=> atp_c + f6p_c + 3 h_c
{'charge': -1.0, 'C': 10.0, 'H': 15.0, 'O': 13.0, 'P': 3.0, 'N': 5}


In [44]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): 2})
print( pgi.reaction )
print( pgi.check_mass_balance() )

g6p_c <=> f6p_c + 2 h_c
{'charge': 2.0, 'H': 2.0}


In [45]:
print( pgi.check_mass_balance() )


{'charge': 2.0, 'H': 2.0}


Tambien podemos ver cuales son las reacciones en las que un determinado metabólito está involucrado:

In [49]:
for rxn in model.metabolites.get_by_id("atp_c").reactions:
    if "BIOMASS" in rxn.id:
        print( rxn )

BIOMASS_Ec_iJO1366_core_53p95M: 0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.000223 2ohph_c + 0.00026 4fe4s_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 54.124831 atp_c + 0.000122 bmocogdp_c + 2e-06 btn_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 2.5e-05 cobalt2_c + 0.133508 ctp_c + 0.000709 cu2_c + 0.09158 cys__L_c + 0.026166 datp_c + 0.027017 dctp_c + 0.027017 dgtp_c + 0.026166 dttp_c + 0.000223 fad_c + 0.006715 fe2_c + 0.007808 fe3_c + 0.26316 gln__L_c + 0.26316 glu__L_c + 0.612638 gly_c + 0.215096 gtp_c + 48.601527 h2o_c + 0.094738 his__L_c + 0.290529 ile__L_c + 0.195193 k_c + 0.019456 kdo2lipid4_e + 0.450531 leu__L_c + 0.343161 lys__L_c + 0.153686 met__L_c + 0.008675 mg2_c + 0.000223 mlthf_c + 0.000691 mn2_c + 7e-06 mobd_c + 0.013894 murein5px4p_p + 0.001831 nad_c + 0.000447 nadp_c + 0.013013 nh4_c + 0.000323 ni2_c + 0.017868 pe160_c + 0.045946 pe160_p + 0.054154 pe161_c + 0.02106 pe161_p + 0.185265 phe__L_c + 0.000223 

# Simulación de flujos metabólicos

Para simular los flujos metabólicos utilizando FBA (optimizamos la velocidad de producción de biomasa:

In [74]:

model = cobra.test.create_test_model("ecoli")
solution=model.optimize()
print( model.summary() )
print( model.objective )

Objective
1.0 BIOMASS_Ec_iJO1366_core_53p95M = 0.9823718127269786

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
     ca2_e     EX_ca2_e  0.005113         0   0.00%
      cl_e      EX_cl_e  0.005113         0   0.00%
 cobalt2_e EX_cobalt2_e 2.456E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0006965         0   0.00%
     fe2_e     EX_fe2_e   0.01578         0   0.00%
  glc__D_e  EX_glc__D_e        10         6 100.00%
       k_e       EX_k_e    0.1918         0   0.00%
     mg2_e     EX_mg2_e  0.008522         0   0.00%
     mn2_e     EX_mn2_e 0.0006788         0   0.00%
    mobd_e    EX_mobd_e 0.0001267         0   0.00%
     nh4_e     EX_nh4_e     10.61         0   0.00%
     ni2_e     EX_ni2_e 0.0003173         0   0.00%
      o2_e      EX_o2_e     17.58         0   0.00%
      pi_e      EX_pi_e    0.9476         0   0.00%
     so4_e     EX_so4_e    0.2478         0   0.00%
     zn2_e     EX_zn2_e  0.000335         0   0.00%

Secretion
---------
Metabolite    

In [54]:
print( model.objective )

Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1


In [62]:
model.medium

{'EX_ca2_e': 1000.0,
 'EX_cbl1_e': 0.01,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [63]:
model.objective=model.reactions.get_by_id("EX_h_e")

In [64]:
print( model.objective )
solution2 = model.optimize()
model.summary()

Maximize
1.0*EX_h_e - 1.0*EX_h_e_reverse_3e0c5


Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,20,1,25.00%
fe3_e,EX_fe3_e,24.99,0,0.00%
glc__D_e,EX_glc__D_e,10,6,75.00%
h2o_e,EX_h2o_e,32.49,0,0.00%
o2_e,EX_o2_e,13.75,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
fe2_e,EX_fe2_e,-24.99,0,0.00%
for_e,EX_for_e,-80,1,100.00%
h_e,EX_h_e,-105,0,0.00%


## Visualización de los resultados


Para visualizar los resultados usaremos la herramienta [Escher](https://escher.readthedocs.io/en/latest/index.html), la cual puedes instalar siguiendo estas [instrucciones](https://anaconda.org/BjornFJohansson/escher)


Para instalar ```escher``` debes abrir el terminal de anaconda (si usas Windowns, busca Anaconda Prompt ) y escribir en el siguiente comando:

```python
pip install escher --user
```

In [65]:
import escher
from escher import Builder

In [66]:
# VISUALIZATION OF RESULTS
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [67]:
builder.reaction_data = solution.fluxes

In [68]:
builder

Builder(reaction_data={'DM_4crsol_c': 0.00021906891423811625, 'DM_5drib_c': 0.0002210336578635664, 'DM_aacald_…

In [75]:
cobra.io.save_json_model(model, "test.json")


## Ejercicio

Simula y los flujos metabólicos de *E. coli* cuando:
    
1. El flujo de consumo de glucosa ("EX_glc__D_e") es igual a -2 [mmol/gDW/h]
2. El limite inferior para el flujo de consumo de oxigeno ("EX_o2_e") es igual a -10 [mmol/gDW/h]

En cada caso ilustra la distribución de flujos usando ```escher```

In [80]:
model = cobra.test.create_test_model("ecoli")

model.reactions.get_by_id("EX_glc__D_e").bounds
model.reactions.get_by_id("EX_glc__D_e").bounds=(-2,-2)
model.reactions.get_by_id("EX_glc__D_e").bounds

(-2, -2)

In [77]:
model.reactions.get_by_id("EX_glc__D_e").bounds=(-2,-2)
model.reactions.get_by_id("EX_glc__D_e").bounds

(-2, -2)

In [81]:
solution1=model.optimize()

In [82]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0009682,0,0.00%
cl_e,EX_cl_e,0.0009682,0,0.00%
cobalt2_e,EX_cobalt2_e,4.651E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001319,0,0.00%
fe2_e,EX_fe2_e,0.002988,0,0.00%
glc__D_e,EX_glc__D_e,2,6,100.00%
k_e,EX_k_e,0.03631,0,0.00%
mg2_e,EX_mg2_e,0.001614,0,0.00%
mn2_e,EX_mn2_e,0.0001285,0,0.00%
mobd_e,EX_mobd_e,2.4E-05,0,0.00%


In [83]:
model = cobra.test.create_test_model("ecoli")

model.reactions.get_by_id("EX_o2_e").bounds
model.reactions.get_by_id("EX_o2_e").bounds=(-10,1000)
model.reactions.get_by_id("EX_o2_e").bounds

(-10, 1000)

In [84]:
solution2=model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0037,0,0.00%
cl_e,EX_cl_e,0.0037,0,0.00%
cobalt2_e,EX_cobalt2_e,1.777E-05,0,0.00%
cu2_e,EX_cu2_e,0.000504,0,0.00%
fe2_e,EX_fe2_e,0.005867,0,0.00%
fe3_e,EX_fe3_e,0.00555,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1388,0,0.00%
mg2_e,EX_mg2_e,0.006167,0,0.00%
mn2_e,EX_mn2_e,0.0004912,0,0.00%
